In [1]:
from dask.distributed import Client
import dask.dataframe as dd
import numpy as np

In [2]:
client = Client(n_workers=1, threads_per_worker=20, processes=True, memory_limit='10GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 20,Total memory: 9.31 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58853,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 9.31 GiB
Comm: tcp://127.0.0.1:58862,Total threads: 20
Dashboard: http://127.0.0.1:58863/status,Memory: 9.31 GiB
Nanny: tcp://127.0.0.1:58856,


In [3]:
df = dd.read_csv("C:/Users/user/Downloads/alms-bd-2022-e5a2b33689f1/alms-bd-2022-e5a2b33689f1/tasks/cik.csv")

In [4]:
def compute_presense(dst):
    return ((dst.normal_ballots + dst.bad_ballots) / dst.total_ballots) * 100

In [5]:
pres_df = df.apply(compute_presense, axis=1, meta=(None, 'float64'))

In [6]:
df['presense'] = pres_df

In [7]:
df.groupby(['regin'])

In [8]:
df_sort = df.sort_values('presense', ascending=False)

In [9]:
# Take the candidate and select the UIK where he received the most ballots
df_uik_max_votes = df_sort[df_sort['registered_voters'] > 300]
df_uik_max_votes_putin = df_uik_max_votes['putin'].max()
value = df_uik_max_votes_putin.compute()
df_uik_max_votes[df_uik_max_votes['putin'] == value].compute()

,regin,tik,uik,registered_voters,total_ballots,early_voters,tik_voters,home_voters,empty_ballots,home_ballots,...,extra_ballots,baburin,grudinin,zhirinovsky,putin,sobchak,suraykin,titov,yavlinsky,presense
92828,Территория за пределами РФ,NaN,УИК №8342,7746,13470,4006,3739,1,5719,4007,...,0,22,235,80,7221,82,34,22,29,57.468448


In [10]:
# Count the maximum difference in the region
df_max_by_reg = df_sort.groupby("regin").presense.agg(['min', 'max', 'mean'])
diff = df_max_by_reg['max'] - df_max_by_reg['min']
df_max_by_reg['difference'] = diff
max_diff = df_max_by_reg['difference'].max().compute()
df_max_by_reg[df_max_by_reg['difference'] == max_diff].compute()

,min,max,mean,difference
regin,,,,
Республика Саха (Якутия),0.0,100.0,74.407687,100.0


In [11]:
df_var = df_sort.drop(columns=['registered_voters', 'total_ballots', 'early_voters', 'tik_voters', 'home_voters', 'empty_ballots',
                                        'home_ballots', 'tik_ballots', 'bad_ballots', 'normal_ballots', 'missing_ballots', 'baburin', 'grudinin',
                                        'zhirinovsky', 'putin', 'sobchak', 'suraykin', 'titov', 'yavlinsky', 'extra_ballots'])
df_var_rename = df_var.rename(columns={"presense":"presense_dispersion"})
df_std = df_var_rename.groupby('regin').std()
df_var = df_var_rename.groupby('regin').var()

In [12]:
df_std.compute()

,presense_dispersion
regin,
Алтайский край,10.511163
Амурская область,12.529601
Архангельская область,14.053107
Астраханская область,12.669397
Белгородская область,11.328166
...,...
Чувашская Республика - Чувашия,11.252229
Чукотский автономный округ,9.564709
Ямало-Ненецкий автономный округ,9.532380


In [13]:
df_var.compute()

,presense_dispersion
regin,
Алтайский край,110.484558
Амурская область,156.990889
Архангельская область,197.489826
Астраханская область,160.513612
Белгородская область,128.327351
...,...
Чувашская Республика - Чувашия,126.612666
Чукотский автономный округ,91.483664
Ямало-Ненецкий автономный округ,90.866273


In [14]:
tot_ball = df_sort.total_ballots.sum().compute()
df_candidates = df_sort.drop(columns=['regin', 'tik', 'uik', 'registered_voters', 'total_ballots', 'early_voters', 'tik_voters', 'home_voters', 'empty_ballots',
                                        'home_ballots', 'tik_ballots', 'bad_ballots', 'normal_ballots', 'missing_ballots', 'extra_ballots', 'presense'])
candidates_array = df_candidates.to_dask_array()
non_zero_elems = np.count_nonzero(candidates_array, axis=0)
sum = np.sum(candidates_array, axis=0)
divided = np.round(np.divide(sum, tot_ball) * 100).astype(int)
print(type(divided))
print(type(non_zero_elems))

<class 'dask.array.core.Array'>
<class 'dask.array.core.Array'>


In [15]:
import dask.array as da
names = np.array(['baburin', 'grudinin','zhirinovsky', 'putin', 'sobchak', 'suraykin', 'titov', 'yavlinsky'])
names_arr = da.from_array(names).astype(str)
divided_arr = divided.astype(str)
non_zero_elems_arr = non_zero_elems.astype(str)
# print(type(names_arr))
# names_arr.compute()
cand = dd.concat([dd.from_dask_array(c) for c in [names_arr, divided, non_zero_elems]], axis = 1)
# cand = dd.concat([names_arr_df, divided_df, non_zero_elems_df], axis=1)
cand.compute()
cand.columns = ["candidate_names", "percentage, %", "amount_of_tik"]
cand.reset_index()
cand.set_index("candidate_names")
cand.compute()

,candidate_names,"percentage, %",amount_of_tik
0,baburin,0,74279
1,grudinin,9,92047
2,zhirinovsky,4,90759
3,putin,54,92894
4,sobchak,1,78478
5,suraykin,0,75527
6,titov,1,68858
7,yavlinsky,1,66150
